In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('../../cii-new/data/20221007_20231006_all.csv')

In [34]:
reads = pd.read_csv('../../citizen-attention-computing/data/study_area_data/read_num.gz')

In [35]:
keywords = pd.read_csv('./keywords.csv')['keyword'].tolist()

In [36]:
keywords

['低碳', '低碳出行', '低碳饮食', '低碳经济', '碳中和', '碳达峰', '碳排放', '碳足迹', '碳交易', '碳汇']

In [37]:
pattern = '|'.join(keywords) 
lowC_texts = df[
        df['text'].str.contains(pattern, na=False) & 
        df['gzh'].str.contains('发布', na=False)
    ][['link', 'title', 'gzh', 'text', 'province']]
lowC_reads = reads[reads.link.isin(lowC_texts['link'])]
study_data = pd.merge(lowC_texts, lowC_reads, on='link')
study_data = study_data.drop_duplicates()

In [38]:
# 多少篇文章
len(study_data)

21463

In [39]:
# 2022-2023年微信公众号包含新闻文章多少座城市
city_names = list(set(study_data['city_name'].tolist()))
cities = pd.DataFrame(city_names, columns=['name'])
len(cities)

268

In [40]:
study_data.to_csv('./data/news.csv', index=None)

## 计算 低碳相关新闻数量/总新闻数量

In [41]:
def get_low_carbon_news_count():
    city_counts = study_data['city_name'].value_counts()
    city_counts_df = city_counts.reset_index()
    city_counts_df.columns = ['city_name', 'carbon_count']
    return city_counts_df

df1 = get_low_carbon_news_count()

In [42]:
def get_all_news_count():
    all_news = df[df.city_name.isin(cities['name'])]
    city_counts = all_news['city_name'].value_counts()
    city_counts_df = city_counts.reset_index()
    city_counts_df.columns = ['city_name', 'total_count']
    return city_counts_df

df2 = get_all_news_count()

In [47]:
# gov_attention = (carbon_count / total_count) * log(total_count)
def output_news_ration():
    temp = pd.merge(df1, df2, on='city_name')
    # temp['gov_attention'] = temp['carbon_count'] / temp['total_count']
    temp['gov_attention'] = (temp['carbon_count'] / temp['total_count']) * np.log(temp['total_count'])
    temp.to_csv('./data/news_tj.csv', index=None)
    return temp

output_news_ration()

,city_name,carbon_count,total_count,gov_attention
0,盐城市,432,2463,1.369690
1,宜宾市,417,3072,1.090021
2,成都市,316,4852,0.552749
3,红河哈尼族彝族自治州,309,5537,0.481007
4,上海市,285,5604,0.438955
...,...,...,...,...
263,桂林市,3,138,0.107114
264,鄂州市,2,172,0.059855
265,长治市,2,48,0.161300
266,崇左市,2,298,0.038236
